# Finding k

You have been analyzing the pricing data on one of the stocks your firm owns. Specifically, you were examining the relationship between the day's trading volume and the spread between the high and low trading price. Use the elbow method to determine the optimal number of clusters, `k`, that should be used to segment these trades. Once the elbow curve has been established, evaluate the two most likely values for `k` using the K-means algorithm and a scatter plot.


In [9]:
# Import the modules
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

## Read in the `stock_data.csv` file from the Resources folder and create a DataFrame. Set the “date” column to create the DatetimeIndex. Be sure to include parameters for `parse_dates` and `infer_datetime_format`.

In [10]:
# Read in the CSV file as a Pandas DataFrame
spread_df = pd.read_csv('../Resources/stock_data.csv')

spread_df.set_index(pd.to_datetime(spread_df['date'], infer_datetime_format=True), inplace=True)
spread_df = spread_df.drop(columns = 'date')

# Review the DataFrame
display(spread_df.head())
display(spread_df.tail())

,close,volume,open,high,low,returns,hi_low_spread
date,,,,,,,
2009-04-30,3.61,18193730,3.55,3.73,3.53,0.028490,0.20
2009-05-01,3.82,16233940,3.55,3.90,3.55,0.058172,0.35
2009-05-04,4.26,21236940,3.90,4.30,3.83,0.115183,0.47
2009-05-05,4.32,16369170,4.36,4.39,4.11,0.014085,0.28
2009-05-06,4.31,15075630,4.45,4.45,4.12,-0.002315,0.33


,close,volume,open,high,low,returns,hi_low_spread
date,,,,,,,
2019-04-23,27.97,41583740,28.18,28.49,27.79,-0.007452,0.70
2019-04-24,28.46,51487330,28.10,28.85,27.93,0.017519,0.92
2019-04-25,27.66,56709000,28.67,28.86,27.36,-0.028110,1.50
2019-04-26,27.88,48736860,27.66,27.90,27.05,0.007954,0.85
2019-04-29,27.69,44265600,27.90,28.14,27.50,-0.006815,0.64


## Create two lists: one to hold the list of inertia scores and another for the range of k values (from 1 to 11) to analyze.

In [11]:
# Create a a list to store inertia values
inertia = []

# Create a a list to store the values of k
k = list(range(1, 11))

## Using a for-loop to evaluate each instance of k, define a K-means model, fit the K-means model based on the scaled DataFrame, and append the model’s inertia to the empty inertia list that you created in the previous step.

In [12]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the spread_df DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k: 
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(spread_df)
    inertia.append(k_model.inertia_)

## Store the values for k and the inertia in a Dictionary called `elbow_data`. Use `elbow_data` to create a Pandas DataFrame called `df_elbow`.

In [13]:
# Create a Dictionary that holds the list values for k and inertia
elbow_data = {'k' : k, 'inertia': inertia}

# Create a DataFrame using the elbow_data Dictionary
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
display(df_elbow.head())
display(df_elbow.tail())

,k,inertia
0,1,2.835703e+18
1,2,9.903144e+17
2,3,4.999629e+17
3,4,3.062100e+17
4,5,2.101772e+17


,k,inertia
5,6,1.540278e+17
6,7,1.101208e+17
7,8,8.440364e+16
8,9,6.657967e+16
9,10,5.220169e+16


## Using hvPlot, plot the `df_elbow` DataFrame to visualize the elbow curve.

In [14]:
# Plot the DataFrame
df_elbow.hvplot.line(
    x = 'k',
    y = 'inertia', 
    title = 'Elbow Curve', 
    xticks = k
)

:Curve   [k]   (inertia)

## Perform the following tasks for each of the two most likely values of `k`:

* Define a K-means model using `k` to define the clusters, fit the model, make predictions, and add the prediction values to a copy of the scaled DataFrame and call it `spread_predictions_df`.

* Plot the clusters. The x-axis should reflect the "hi_low_spread", and the y-axis should reflect the "close" price.

In [16]:
# Define the model with the lower value of k clusters
# Use a random_state of 1 to generate the model
model = KMeans(n_clusters=1, random_state=1)

# Fit the model
model.fit(spread_df)

# Make predictions
k_lower =  model.predict(spread_df)

# Create a copy of the DataFrame and name it as spread_df_predictions
spread_df_predictions = spread_df.copy()

# Add a class column with the labels to the spread_df_predictions DataFrame
spread_df_predictions['clusters_lower'] = k_lower

In [22]:
# Plot the clusters
spread_df_predictions.hvplot.scatter(
    x = 'hi_low_spread', 
    y = 'close', 
    by = 'clusters_lower'
)

:NdOverlay   [clusters_lower]
   :Scatter   [hi_low_spread]   (close)

In [20]:
# Define the model with the higher value of k clusters
# Use a random_state of 1 to generate the model
model =  KMeans(n_clusters=10, random_state=1)

# Fit the model
model.fit(spread_df)

# Make predictions
k_higher =  model.predict(spread_df)

# Add a class column with the labels to the spread_df_predictions DataFrame
spread_df_predictions['clusters_higher'] = k_higher

In [23]:
# Plot the clusters
spread_df_predictions.hvplot.scatter(
    x = 'hi_low_spread', 
    y = 'close', 
    by = 'clusters_higher'
)

:NdOverlay   [clusters_higher]
   :Scatter   [hi_low_spread]   (close)

## Answer the following question

* Considering the plot, what’s the best number of clusters to choose, or value of k?
    *  > Using the higher `k` value, we can see more distinguished scatter plots that can help determine the optimal number of clusters, `k`, that should be used to   segment these trades; looking at the scatter plot we can see that the red cluster (7) and the light blue cluster (5) are trades that have closed at a lower trading price while the other clusters seem to be more spread out and have closed at a higher trading price. The scatter plot indicates that most people sell when the high-low spread is at a minimum. 